<a href="https://colab.research.google.com/github/samina-if/AdventOfCode2024/blob/main/Advent_of_Code_Day20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Part 1
from collections import deque

def read_input(file_path):
    """Reads the input file and returns the map as a list of strings."""
    with open(file_path, 'r') as f:
        return [line.strip() for line in f]

def find_positions(grid):
    """Finds the start (S) and end (E) positions in the grid."""
    start = end = None
    for r, row in enumerate(grid):
        for c, char in enumerate(row):
            if char == 'S':
                start = (r, c)
            elif char == 'E':
                end = (r, c)
    return start, end

def is_within_bounds(pos, grid):
    """Checks if a position is within the bounds of the grid."""
    r, c = pos
    return 0 <= r < len(grid) and 0 <= c < len(grid[0])

def get_neighbors(pos, grid):
    """Returns the valid neighbors of a position."""
    r, c = pos
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    for dr, dc in directions:
        nr, nc = r + dr, c + dc
        if is_within_bounds((nr, nc), grid):
            yield (nr, nc)

def bfs(grid, start, end, cheat=False):
    """Performs BFS to find the shortest path, optionally allowing a cheat."""
    queue = deque([(start, 0, False)])  # (position, time, has_cheated)
    visited = set()
    visited.add((start, False))

    while queue:
        pos, time, has_cheated = queue.popleft()

        if pos == end:
            return time

        for neighbor in get_neighbors(pos, grid):
            r, c = neighbor

            if grid[r][c] == '#' and not has_cheated and cheat:
                # Allow cheating through walls
                for extra in get_neighbors((r, c), grid):
                    if grid[extra[0]][extra[1]] == '.' and extra not in visited:
                        visited.add((extra, True))
                        queue.append((extra, time + 1, True))

            elif grid[r][c] != '#' and (neighbor, has_cheated) not in visited:
                visited.add((neighbor, has_cheated))
                queue.append((neighbor, time + 1, has_cheated))

    return float('inf')

def count_cheats(grid, start, end):
    """Counts the number of cheats and groups them by the amount of time saved."""
    normal_time = bfs(grid, start, end, cheat=False)

    cheat_savings = {}
    for r, row in enumerate(grid):
        for c, char in enumerate(row):
            if char == '#':
                # Simulate a cheat starting at this wall
                new_time = bfs(grid, start, end, cheat=True)
                savings = normal_time - new_time
                if savings > 0:
                    cheat_savings[savings] = cheat_savings.get(savings, 0) + 1

    return cheat_savings

# Load the input file
grid = read_input('input.txt')
start, end = find_positions(grid)

# Count cheats and filter those saving at least 100 picoseconds
cheat_savings = count_cheats(grid, start, end)
cheats_100_or_more = sum(count for save, count in cheat_savings.items() if save >= 100)

print(f"Cheats saving at least 100 picoseconds: {cheats_100_or_more}")


Cheats saving at least 100 picoseconds: 10504


In [ ]:
import heapq

def parse_input(file_path):
    with open(file_path, 'r') as file:
        grid = [list(line.strip()) for line in file.readlines()]
    return grid

def find_start_and_end(grid):
    start = None
    end = None
    for y, row in enumerate(grid):
        for x, cell in enumerate(row):
            if cell == 'S':
                start = (x, y)
            elif cell == 'E':
                end = (x, y)
    return start, end

def neighbors(x, y, grid):
    moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    for dx, dy in moves:
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(grid[0]) and 0 <= ny < len(grid):
            yield nx, ny

def shortest_path(grid, start, end, allow_cheat=False):
    rows, cols = len(grid), len(grid[0])
    visited = set()
    heap = [(0, start[0], start[1], 0)]  # (time, x, y, cheats_used)

    while heap:
        time, x, y, cheats_used = heapq.heappop(heap)

        if (x, y, cheats_used) in visited:
            continue
        visited.add((x, y, cheats_used))

        if (x, y) == end:
            return time

        for nx, ny in neighbors(x, y, grid):
            if grid[ny][nx] == '#':  # Wall
                if cheats_used < 2 and allow_cheat:
                    heapq.heappush(heap, (time + 1, nx, ny, cheats_used + 1))
            else:
                heapq.heappush(heap, (time + 1, nx, ny, cheats_used))
    return float('inf')

def calculate_cheat_savings(grid, start, end):
    baseline = shortest_path(grid, start, end, allow_cheat=False)
    cheat_savings = []

    rows, cols = len(grid), len(grid[0])
    for y in range(rows):
        for x in range(cols):
            if grid[y][x] == '#':
                for ny, nx in neighbors(x, y, grid):
                    if grid[ny][nx] == '#':
                        grid[y][x] = '.'
                        grid[ny][nx] = '.'
                        new_time = shortest_path(grid, start, end, allow_cheat=True)
                        savings = baseline - new_time
                        if savings > 0:
                            cheat_savings.append(savings)
                        grid[y][x] = '#'
                        grid[ny][nx] = '#'

    return cheat_savings

def main():
    input_file = 'input.txt'
    grid = parse_input(input_file)
    start, end = find_start_and_end(grid)
    cheat_savings = calculate_cheat_savings(grid, start, end)
    cheats_over_100 = len([s for s in cheat_savings if s >= 100])
    print(f"Number of cheats that save at least 100 picoseconds: {cheats_over_100}")

# Run the main function
if __name__ == "__main__":
    main()


In [2]:
#Part 2
from collections import deque
from typing import List, Set, Tuple, Dict
import heapq

def read_input(filename: str) -> List[str]:
    with open(filename, 'r') as f:
        return [line.strip() for line in f.readlines()]

def find_start_end(grid: List[str]) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    start = end = None
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 'S':
                start = (i, j)
            elif grid[i][j] == 'E':
                end = (i, j)
    return start, end

def get_neighbors(pos: Tuple[int, int], grid: List[str]) -> List[Tuple[int, int]]:
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    neighbors = []
    for dy, dx in directions:
        new_y, new_x = pos[0] + dy, pos[1] + dx
        if (0 <= new_y < len(grid) and
            0 <= new_x < len(grid[0]) and
            grid[new_y][new_x] != '#'):
            neighbors.append((new_y, new_x))
    return neighbors

def shortest_path(grid: List[str], start: Tuple[int, int], end: Tuple[int, int]) -> Dict[Tuple[int, int], int]:
    distances = {}
    queue = [(0, start)]
    distances[start] = 0

    while queue:
        dist, current = heapq.heappop(queue)

        if dist > distances[current]:
            continue

        for next_pos in get_neighbors(current, grid):
            new_dist = dist + 1

            if next_pos not in distances or new_dist < distances[next_pos]:
                distances[next_pos] = new_dist
                heapq.heappush(queue, (new_dist, next_pos))

    return distances

def find_cheats(grid: List[str], normal_distances: Dict[Tuple[int, int], int],
                start: Tuple[int, int], end: Tuple[int, int]) -> Dict[int, int]:
    height, width = len(grid), len(grid[0])
    savings = {}

    # For each possible cheat start position
    for y1 in range(height):
        for x1 in range(width):
            if grid[y1][x1] == '#':
                continue
            pos1 = (y1, x1)
            if pos1 not in normal_distances:
                continue

            # For each possible cheat end position
            for y2 in range(height):
                for x2 in range(width):
                    if grid[y2][x2] == '#':
                        continue
                    pos2 = (y2, x2)

                    # Calculate Manhattan distance between start and end of cheat
                    manhattan_dist = abs(y2 - y1) + abs(x2 - x1)
                    if manhattan_dist > 2:  # Cheat can only last 2 moves
                        continue

                    # If we can reach both positions in normal path
                    if pos1 in normal_distances and pos2 in normal_distances:
                        # Calculate time saved
                        normal_time = normal_distances[end]
                        cheat_time = (normal_distances[pos1] +
                                    manhattan_dist +
                                    (normal_distances[end] - normal_distances[pos2]))

                        if cheat_time < normal_time:
                            saved = normal_time - cheat_time
                            savings[saved] = savings.get(saved, 0) + 1

    return savings

def solve(grid: List[str]) -> int:
    start, end = find_start_end(grid)

    # Replace S and E with . for easier processing
    grid = [row.replace('S', '.').replace('E', '.') for row in grid]

    # Find normal shortest path distances
    normal_distances = shortest_path(grid, start, end)

    # Find all possible cheats and their time savings
    savings = find_cheats(grid, normal_distances, start, end)

    # Count cheats that save at least 100 picoseconds
    return sum(count for saved, count in savings.items() if saved >= 100)

def main():
    grid = read_input("input.txt")
    result = solve(grid)
    print(f"Number of cheats saving at least 100 picoseconds: {result}")

if __name__ == "__main__":
    main()

Number of cheats saving at least 100 picoseconds: 1343


In [4]:
from collections import deque
from typing import List, Set, Tuple, Dict
import heapq

def read_input(filename: str) -> List[str]:
    """
    Reads the racetrack grid from the input file.
    """
    with open(filename, 'r') as f:
        return [line.strip() for line in f.readlines()]

def find_start_end(grid: List[str]) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    """
    Finds the start (S) and end (E) positions in the grid.
    """
    start = end = None
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 'S':
                start = (i, j)
            elif grid[i][j] == 'E':
                end = (i, j)
    return start, end

def get_neighbors(pos: Tuple[int, int], grid: List[str], cheating: bool = False) -> List[Tuple[int, int]]:
    """
    Generates the valid neighbors of a position.
    Allows movement through walls if cheating is enabled.
    """
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    neighbors = []
    for dy, dx in directions:
        new_y, new_x = pos[0] + dy, pos[1] + dx
        if (0 <= new_y < len(grid) and
            0 <= new_x < len(grid[0]) and
            (cheating or grid[new_y][new_x] != '#')):
            neighbors.append((new_y, new_x))
    return neighbors

def find_reachable_positions(grid: List[str], start: Tuple[int, int], max_steps: int) -> Dict[Tuple[int, int], int]:
    """
    Finds all positions reachable from the start position within a given step limit.
    """
    distances = {start: 0}
    queue = [(0, start)]

    while queue:
        dist, current = heapq.heappop(queue)

        if dist > max_steps:
            continue

        for next_pos in get_neighbors(current, grid, cheating=True):
            new_dist = dist + 1

            if new_dist <= max_steps and (next_pos not in distances or new_dist < distances[next_pos]):
                distances[next_pos] = new_dist
                heapq.heappush(queue, (new_dist, next_pos))

    return distances

def shortest_path(grid: List[str], start: Tuple[int, int], end: Tuple[int, int]) -> Dict[Tuple[int, int], int]:
    """
    Computes the shortest path from start to end using Dijkstra's algorithm.
    """
    distances = {}
    queue = [(0, start)]
    distances[start] = 0

    while queue:
        dist, current = heapq.heappop(queue)

        if dist > distances[current]:
            continue

        for next_pos in get_neighbors(current, grid):
            new_dist = dist + 1

            if next_pos not in distances or new_dist < distances[next_pos]:
                distances[next_pos] = new_dist
                heapq.heappush(queue, (new_dist, next_pos))

    return distances

def find_cheats(grid: List[str], normal_distances: Dict[Tuple[int, int], int],
                start: Tuple[int, int], end: Tuple[int, int]) -> Dict[int, int]:
    """
    Identifies all cheats and calculates the time savings for each.
    """
    MAX_CHEAT_LENGTH = 20
    savings = {}
    height, width = len(grid), len(grid[0])

    # For each possible cheat start position
    for y1 in range(height):
        for x1 in range(width):
            if grid[y1][x1] == '#':
                continue
            pos1 = (y1, x1)
            if pos1 not in normal_distances:
                continue

            # Find all positions reachable within MAX_CHEAT_LENGTH steps while cheating
            reachable = find_reachable_positions(grid, pos1, MAX_CHEAT_LENGTH)

            # For each reachable position that's on a valid path
            for pos2, cheat_length in reachable.items():
                if grid[pos2[0]][pos2[1]] == '#':
                    continue

                # If we can reach both positions in normal path
                if pos1 in normal_distances and pos2 in normal_distances:
                    # Calculate time saved
                    normal_time = normal_distances[end]
                    cheat_time = (normal_distances[pos1] +
                                  cheat_length +
                                  (normal_distances[end] - normal_distances[pos2]))

                    if cheat_time < normal_time:
                        saved = normal_time - cheat_time
                        savings[saved] = savings.get(saved, 0) + 1

    return savings

def solve(grid: List[str]) -> int:
    """
    Solves the problem to find the number of cheats saving at least 100 picoseconds.
    """
    start, end = find_start_end(grid)

    # Replace S and E with . for easier processing
    grid = [row.replace('S', '.').replace('E', '.') for row in grid]

    # Find normal shortest path distances
    normal_distances = shortest_path(grid, start, end)

    # Find all possible cheats and their time savings
    savings = find_cheats(grid, normal_distances, start, end)

    # Count cheats that save at least 100 picoseconds
    return sum(count for saved, count in savings.items() if saved >= 100)

def main():
    grid = read_input("input.txt")
    result = solve(grid)
    print(f"Number of cheats saving at least 100 picoseconds: {result}")

if __name__ == "__main__":
    main()


Number of cheats saving at least 100 picoseconds: 982891
